$\textbf{IMPORTS}$

In [15]:
import requests
import numpy as np
import csv
import json
import time
import tqdm
import os
import pprint
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$, BOLIGA.DK

In [2]:
#1.Load pages boliga.dk
links = []
for page in range(1,146): #check the number of pages in link before running this code!!
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{LOGGING FUNCTION}$

In [3]:
# Define the log function to gather the log information
def log(response,logfile,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','output_file']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{output_path}' + "\n") #Append the information and jump to new line

$\textbf{SCRAPE DATA FROM EACH PAGE}$, BOLIGA.DK

In [4]:
#1, Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

log_boliga = 'log_boliga.csv'
for i in links:
    response = requests.get(i, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(",",1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text.split("k",1)[0] for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
     #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"
    time.sleep(0.5) #Sleep for 0.5 seconds
    log(response,log_boliga)

$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [5]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Pris": price_final, "address": address_final, "zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data = pd.DataFrame(df_addzip)

#2. save dataset as v1
df_data.to_csv("v1.csv")
df_data

,Pris,address,zipcode,m2,rooms
0,1.795.000,Kvintus Allé 3,2300 København S,38,1
1,2.595.000,Skelhøjvej 9,2800 Kongens Lyngby,67,3
2,3.595.000,Herninggade 5,2100 København Ø,64,0
3,3.698.000,Vestre Ringgade 226,8000 Aarhus C,100,3
4,3.995.000,Digevej 38,2300 København S,87,3
...,...,...,...,...,...
7142,3.895.000,Promenaden 22,7000 Fredericia,128,3
7143,3.895.000,Promenaden 30,7000 Fredericia,127,3
7144,3.995.000,Promenaden 29,7000 Fredericia,141,4
7145,4.075.000,Promenaden 32,7000 Fredericia,127,3


$\textbf{GET  DISTANCES}, \ dataforsyningen.dk, \ hvorlangterder.dk$

In [6]:
## 1. Load pages
Address_zipcode = df_data["address"] + " " + df_data["zipcode"]
add_zip = []
log_dataforsyning = 'log_dataforsyning.csv'
for i in Address_zipcode:
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={i}&struktur=mini", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    res_j = response_dist.json()
    add_zip.append(res_j)
    time.sleep(0.5) #Sleep for 0.5 seconds
    log(response_dist,log_dataforsyning)

Before continuing, locate missing values due to typing error in column "address" in df_data

In [7]:
#2. Locate missing values
df_add_zip = pd.DataFrame(add_zip)
index = df_add_zip[df_add_zip[0].isnull()][0]

#3. Store index numbers in order to del the corresponding indexes in df_data
index_list = list(index.index.values)

#4. Delete corresponding rows in df_data
for i in index_list:
    df_add_zip.drop(i, inplace=True)

#5. reset_index keeps failing, so here comes another index-method    
df_add_zip["new_index"] = range(len(df_add_zip))
df_add_zip.set_index("new_index", inplace=True)

Drop missing values in df_data

In [8]:
#6. Delete corresponding rows in df_data
for i in index_list:
    df_data.drop(i, inplace=True)

#reset_index keeps failing, so here comes another index-method    
df_data["new_index"] = range(len(df_data))
df_data.set_index("new_index", inplace=True)

In [9]:
#7. save dataset with dropped missing values as v2
df_data.to_csv("v2.csv")
df_data
#Vi mister 151 observationer efter fjernelse af ovenstående obs

,Pris,address,zipcode,m2,rooms
new_index,,,,,
0,1.795.000,Kvintus Allé 3,2300 København S,38,1
1,2.595.000,Skelhøjvej 9,2800 Kongens Lyngby,67,3
2,3.595.000,Herninggade 5,2100 København Ø,64,0
3,3.698.000,Vestre Ringgade 226,8000 Aarhus C,100,3
4,3.995.000,Digevej 38,2300 København S,87,3
...,...,...,...,...,...
6994,1.600.000,Skiftesporet 7,8930 Randers NØ,68,2
6995,7.750.000,Weidekampsgade 65,2300 København S,152,4
6996,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3


In [10]:
#8. Get longitude, x, and latitudes, y from address i
x = []
y = []

for i in range(len(df_add_zip)):
    xx = df_add_zip[0][i]["x"]
    x.append(xx)

for i in range(len(df_add_zip)):
    yy = df_add_zip[0][i]["y"]
    y.append(yy)

In [11]:
#9. Add coordinates to df_data
df_data["x"] = x
df_data["y"] = y
xy = df_data["y"].apply(str) +","+ df_data["x"].apply(str)

In [16]:
#10 Get distance to nearest x
daycare_distance = []
log_dist = 'log_dist'
for i in tqdm.tqdm(xy):
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=daycare&decodepolyline',headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    afstand_daycare_json.json()
    key = afstand_daycare_json.json()['daycare']
    key = key['routedmeters']
    daycare_distance.append(key)
    time.sleep(0.5) #Sleep for 0.5 seconds
    log(afstand_daycare_json,log_dist)

100%|██████████| 6999/6999 [1:12:59<00:00,  1.60it/s]


In [17]:
regionaltog_distance = []

for i in tqdm.tqdm(xy):
    afstand_regionaltog_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=train&decodepolyline', headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    afstand_regionaltog_json.json()
    key = afstand_regionaltog_json.json()['train']
    key = key['routedmeters']
    regionaltog_distance.append(key)
    time.sleep(0.5) #Sleep for 0.5 seconds

100%|██████████| 6999/6999 [1:18:30<00:00,  1.49it/s]


In [18]:
school_distance = []

for i in tqdm.tqdm(xy):
    afstand_school_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=school&decodepolyline',headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    afstand_school_json.json()
    key = afstand_school_json.json()['school']
    key = key['routedmeters']
    school_distance.append(key)
    time.sleep(0.5) #Sleep for 0.5 seconds

100%|██████████| 6999/6999 [1:18:38<00:00,  1.48it/s]


In [19]:
supermarket_distance = []

for i in tqdm.tqdm(xy):
    afstand_supermarket_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=supermarket&decodepolyline',headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    afstand_supermarket_json.json()
    key = afstand_supermarket_json.json()['supermarket']
    key = key['routedmeters']
    supermarket_distance.append(key)
    time.sleep(0.5) #Sleep for 0.5 seconds

100%|██████████| 6999/6999 [1:18:57<00:00,  1.48it/s]


In [20]:
hospital_distance = []

for i in tqdm.tqdm(xy):
    hospital_distance_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=hospital&decodepolyline',headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
    hospital_distance_json.json()
    key = hospital_distance_json.json()['hospital']
    key = key['routedmeters']
    hospital_distance.append(key)
    time.sleep(0.5) #Sleep for 0.5 seconds

100%|██████████| 6999/6999 [1:18:56<00:00,  1.48it/s]


In [21]:
#4 Add coordinates of addresses in dataframe
df_data["daycare_distance"] = daycare_distance
df_data["train_distance"] = regionaltog_distance
df_data["school_distance"] = school_distance
df_data["supermarket_distance"] = supermarket_distance
df_data["hospital_distance"] = hospital_distance

In [22]:
#11. save dataset with distances
df_data.to_csv("v3.csv")
df_data

,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance
new_index,,,,,,,,,,,,
0,1.795.000,Kvintus Allé 3,2300 København S,38,1,12.619615,55.660817,371.50,4240.07,643.80,226.93,847.41
1,2.595.000,Skelhøjvej 9,2800 Kongens Lyngby,67,3,12.482367,55.763944,456.10,2358.62,831.28,581.67,6374.42
2,3.595.000,Herninggade 5,2100 København Ø,64,0,12.588390,55.707378,288.63,2095.23,638.95,263.23,2556.49
3,3.698.000,Vestre Ringgade 226,8000 Aarhus C,100,3,10.196700,56.169798,471.54,2625.79,704.16,1028.13,4453.36
4,3.995.000,Digevej 38,2300 København S,87,3,12.587169,55.647657,431.08,2318.79,679.94,1069.28,2960.85
...,...,...,...,...,...,...,...,...,...,...,...,...
6994,1.600.000,Skiftesporet 7,8930 Randers NØ,68,2,10.064362,56.469382,569.88,3445.72,742.10,476.70,1699.02
6995,7.750.000,Weidekampsgade 65,2300 København S,152,4,12.580813,55.667745,264.35,1851.89,804.97,333.11,4025.06
6996,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71


$\textbf{CONSTRUCT A VARIABEL FOR REGION}$

In [23]:
#1. Load API
response_reg = requests.get(f"https://api.dataforsyningen.dk/regioner", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk', "note": "I am a student extracting data from this webpage. If you have any problem or concern, email me."})
res_reg = response_reg.json()

In [24]:
#2. Get the coordinate intervals for each region
regions = []

for i in range(0,5):
    rg = res_reg[i]["bbox"]
    regions.append(rg)
regions

[[8.18951662, 56.53454667, 11.22599137, 57.76025478],
 [8.07887623, 55.64437916, 11.66419061, 56.84325702],
 [8.06320315, 54.71828064, 10.99555236, 55.95324986],
 [11.60211644, 54.98354657, 15.31831444, 56.20520188],
 [10.8148054, 54.54440636, 12.64551576, 56.0173058]]

In [25]:
#3. Use the coordinate intervals above to place each observation in a region
def region(df_data):
    
    if (8.18951662 < df_data['x'] < 11.22599137) and (56.53454667 < df_data['y'] < 57.76025478):  
        return 'Nordjylland'
    elif (8.07887623 <  df_data['x'] < 11.66419061) and (55.64437916 < df_data['y'] < 56.84325702): 
        return 'Midtjylland'
    elif (8.06320315 < df_data['x'] < 10.99555236) and (54.71828064 < df_data['y'] < 55.95324986):  
        return 'Syddanmark'
    elif (11.60211644 < df_data['x'] < 15.31831444) and (54.98354657 < df_data['y'] < 56.20520188):  
        return 'Hovedstaden'
    elif (10.8148054 < df_data['x'] < 12.64551576) and (54.54440636 < df_data['y'] < 56.0173058):  
        return 'Sjælland'
df_data['Region'] = df_data.apply(region, axis=1)
#display(df_data)

In [26]:
#4. save dataset including regions
df_data.to_csv("v4.csv")
df_data

,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance,Region
new_index,,,,,,,,,,,,,
0,1.795.000,Kvintus Allé 3,2300 København S,38,1,12.619615,55.660817,371.50,4240.07,643.80,226.93,847.41,Hovedstaden
1,2.595.000,Skelhøjvej 9,2800 Kongens Lyngby,67,3,12.482367,55.763944,456.10,2358.62,831.28,581.67,6374.42,Hovedstaden
2,3.595.000,Herninggade 5,2100 København Ø,64,0,12.588390,55.707378,288.63,2095.23,638.95,263.23,2556.49,Hovedstaden
3,3.698.000,Vestre Ringgade 226,8000 Aarhus C,100,3,10.196700,56.169798,471.54,2625.79,704.16,1028.13,4453.36,Midtjylland
4,3.995.000,Digevej 38,2300 København S,87,3,12.587169,55.647657,431.08,2318.79,679.94,1069.28,2960.85,Hovedstaden
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,1.600.000,Skiftesporet 7,8930 Randers NØ,68,2,10.064362,56.469382,569.88,3445.72,742.10,476.70,1699.02,Midtjylland
6995,7.750.000,Weidekampsgade 65,2300 København S,152,4,12.580813,55.667745,264.35,1851.89,804.97,333.11,4025.06,Hovedstaden
6996,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71,Hovedstaden


$\textbf{CLEAN DATASET}$

In [27]:
#Clean dataset and save as v5

df_data = df_data.drop_duplicates().reset_index()
del df_data["new_index"]

df_data.Pris = df_data.Pris.astype(str)
df_data.Pris = df_data.Pris.str.replace('.', '', regex=True)

df_data["Pris"] = df_data["Pris"].apply(pd.to_numeric, errors='coerce')

df_data.to_csv("v5.csv")

In [2]:
df_data = pd.read_csv('v5.csv', index_col=[0])

In [1]:
df_data = pd.get_dummies(df_data, columns=['Region'])

NameError: name 'pd' is not defined

In [ ]:
df_data["Pris"] = df_data["Pris"].str.rstrip('.')

In [ ]:
df_data["Pris"] = df_data["Pris"].str.rstrip('.')

In [ ]:
x

In [ ]:
y

In [ ]:
y = np.array(y, dtype=np.float64)
x = np.array(x, dtype=np.float64)

In [2]:
# splitting into development (2/3) and test data (1/3)
x_dev, x_test, y_dev, y_test = train_test_split(x, y, test_size=1/3, random_state=0)

# splitting development into train (1/3) and validation (1/3)
x_train, x_val, y_train, y_val = train_test_split(x_dev, y_dev, test_size=1/2, random_state=52)

lambda_ = np.logspace(-1, 1, 22)
l1_ratio_ = np.logspace(0, 3, 22)
tol = 0.001

NameError: name 'train_test_split' is not defined

**OLS**

In [ ]:
pipe_OLS = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           LinearRegression())
                         
pipe_OLS.fit(X_dev, y_dev)
print(mse(pipe_OLS.predict(X_test),y_test), round(np.sqrt(mse(pipe_OLS.predict(X_test),y_test))))

**LASSO**

In [ ]:
pipe_lasso = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Lasso(max_iter = 10000, alpha=lambda_, tol=tol))

train_scores, test_scores = validation_curve(estimator=pipe_lasso,
                                             X=X_train, y=y_train,
                                             param_name='lasso__alpha',
                                             param_range=lambda_,
                                             scoring='neg_mean_squared_error',
                                             cv=5)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':lambda_}).set_index('lambda')   

optimal_lambda_lasso = mse_score.Validation.nsmallest(1)
lambda_o_ = optimal_lambda_lasso.index



In [ ]:
pipe_lasso.fit(X_dev, y_dev)
print(lambda_o_,' ',round(mse(pipe_lasso.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_lasso.predict(X_test),y_test)),3))

**RIDGE**

In [ ]:
pipe_ridge = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Ridge(alpha = l1_ratio_, tol=tol))

train_scores, test_scores = validation_curve(estimator=pipe_ridge,
                                             X=X_train, y=y_train,
                                             param_name='ridge__alpha',
                                             param_range=l1_ratio_,
                                             scoring='neg_mean_squared_error',
                                             cv=5)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':l1_ratio_}).set_index('lambda')   

lambda_o_ = mse_score.Validation.nsmallest(1)
lambda_o_ = lambda_o_.index

In [ ]:
pipe_ridge.fit(X_dev, y_dev)
print(lambda_o_,' ', round(mse(pipe_ridge.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_ridge.predict(X_test),y_test)),3))

**ELASTIC NET**

In [ ]:
pipe_el = make_pipeline(PolynomialFeatures(include_bias=False), 
                        StandardScaler(),
                        ElasticNet(tol=.01))

grids = GridSearchCV(estimator=pipe_el, 
                  param_grid={'elasticnet__alpha':np.logspace(-1,1,22),
                              'elasticnet__l1_ratio':np.linspace(0,3,10)}, 
                  scoring='neg_mean_squared_error', 
                  n_jobs=8,
                  iid=False,
                  cv=10)

gs.fit(X_train, y_train)
gs.predict(X_train)

In [ ]:
print(grids.best_params_)
print(lambda_o_,' ', round(mse(gs.predict(X_test),y_test),3), round(np.sqrt(mse(gs.predict(X_test),y_test)),3))